In [24]:
from telnetlib import X3PAD
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 

from sklearn.utils import shuffle
import pickle 
import sqlite3


In [25]:
df_2020 = pd.read_csv('merged_gw2020.csv',low_memory = False)
df_2021 = pd.read_csv('merged_gw2021.csv',low_memory = False)


In [26]:
season_2020 = pd.DataFrame({'season' : ['2019/20']*df_2020.shape[0]   })
season_2021 = pd.DataFrame({'season' : ['2020/21']*df_2021.shape[0]   })

In [27]:
df_2020 =pd.concat([df_2020, season_2020] ,axis = 'columns')
df_2021 =pd.concat([df_2021, season_2021] ,axis = 'columns')


In [28]:
df= pd.concat([df_2020 , df_2021], axis = 'index',ignore_index = True)



In [29]:
df["was_home"] = df["was_home"].astype(int)


In [30]:

df.sort_values(by=['name','season','GW'], inplace =True)
df.reset_index(inplace = True)


df.drop(['index'],axis= 'columns' , inplace =True)

In [31]:
df['id'] = df.groupby(['name']).ngroup()

In [32]:
df['pos']= df.groupby(['position']).ngroup()

In [33]:
df['teams'] = df.groupby(['team']).ngroup()

In [34]:
identity = df['id'].unique()


### points milaune thau


In [35]:
#enter value of x where x is the latest gameweek
x = 25

In [36]:
filt = (df['GW'] == x) & (df['season']=='2020/21')
df_points = df.loc[filt , ['total_points','id' ]]
df_points.reset_index(inplace = True)


df_points.drop(['index'],axis= 'columns' , inplace =True)
df_points

,total_points,id
0,0,0
1,0,0
2,0,1
3,2,2
4,0,5
...,...,...
695,1,957
696,2,957
697,0,959
698,0,962


In [37]:
y=x-1
Gameweeks=[]

In [38]:
for i in range(5):
    Gameweeks.append(y)
    y=y-1


In [39]:
dataframes = {}

In [40]:
for GW in Gameweeks:
    filt=(df['GW'] == GW) & (df['season'] == '2020/21')
    dataframes[GW] = df[filt]
    dataframes[GW].reset_index(inplace = True)
    dataframes[GW].drop(['index'],axis= 'columns' , inplace =True)
    dataframes[GW].drop([ 'kickoff_time','season', 'name' , 'position','pos' ,'GW' ,'team','teams'],axis= 'columns' , inplace =True)
    

/home/aavash/Downloads/jupyter/environment/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
dataframes[24].columns

Index(['xP', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'id'],
      dtype='object')

In [42]:
n = 0
for keys in dataframes:
    column = []
    
    for value in dataframes[keys].columns:
        
        if (value != 'id')  & (value!='team'):
            value = value +f'_{n}' 
        column.append(value)
        
    dataframes[keys].columns = column
    n=n-1       

In [45]:
for keys in dataframes:
    dataframes[keys].drop_duplicates(subset='id' , keep ='first',inplace = True)

/home/aavash/Downloads/jupyter/environment/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [46]:
for keys in dataframes:
    print(dataframes[keys]['id'].nunique())
    print(dataframes[keys].shape)
    

634
(634, 32)
696
(696, 32)
622
(622, 32)
617
(617, 32)
460
(460, 32)


In [47]:
dataframes[23].to_csv('check.csv')

In [48]:

df1 = pd.merge(dataframes[x-1] , dataframes[x-2], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-3], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-4], on = 'id' , how = 'outer')
df1 = pd.merge(df1 , dataframes[x-5], on = 'id' , how = 'outer')


In [49]:
filt = (df['GW'] == x-1) | (df['GW'] == x-2) | (df['GW'] == x-3) | (df['GW'] == x-4) | (df['GW'] == x-5)
df_position = df.loc[filt , ['id' , 'pos','teams']]

In [50]:
df_position.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_position.reset_index(inplace = True)
df_position.drop(['index'],axis= 'columns' , inplace =True)

In [51]:
df1.drop_duplicates(subset='id',keep = 'first',inplace = True)
df1.reset_index(inplace = True)
df1.drop(['index'],axis= 'columns' , inplace =True)

In [52]:
df1 = pd.merge(df1 , df_position, on = 'id' , how = 'inner')

In [58]:
x


25

In [54]:
filt = df['GW']== x
df_points = df.loc[filt,['id' , 'total_points']]
df_points.drop_duplicates(subset='id',keep = 'first',inplace = True)
df_points.reset_index(inplace = True)
df_points.drop(['index'],axis= 'columns' , inplace =True)

In [55]:
df_points

,id,total_points
0,0,0
1,1,2
2,2,2
3,3,0
4,4,3
...,...,...
911,960,0
912,961,0
913,962,0
914,963,2


In [ ]:

df1 = pd.merge(df1 , df_points, on = 'id' , how = 'inner')


,xP_0,assists_0,bonus_0,bps_0,clean_sheets_0,creativity_0,element_0,fixture_0,goals_conceded_0,goals_scored_0,...,threat_-4,total_points_-4,transfers_balance_-4,transfers_in_-4,transfers_out_-4,value_-4,was_home_-4,yellow_cards_-4,pos,teams
0,6.5,0.0,2.0,30.0,1.0,25.8,411.0,235.0,0.0,0.0,...,0.0,0.0,-26734.0,479.0,27213.0,53.0,0.0,0.0,0,21
1,1.3,0.0,0.0,5.0,0.0,0.8,575.0,233.0,0.0,0.0,...,24.0,7.0,-3.0,25.0,28.0,49.0,0.0,0.0,3,4
2,9.0,0.0,2.0,31.0,1.0,1.0,559.0,236.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,16
3,0.0,0.0,0.0,0.0,0.0,0.0,496.0,231.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1
4,1.0,0.0,0.0,0.0,0.0,0.0,288.0,233.0,0.0,0.0,...,0.0,2.0,-1478.0,2307.0,3785.0,51.0,1.0,0.0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,-81388.0,2479.0,83867.0,56.0,1.0,0.0,0,5
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,-3728.0,158.0,3886.0,86.0,1.0,0.0,1,5
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.0,1.0,4822.0,8551.0,3729.0,47.0,1.0,0.0,0,5


In [ ]:
df1 = df1.fillna(df1.median())

In [ ]:
mean = df1['total_points'].mean()
std = df['total_points'].std()

In [ ]:
df1

In [ ]:
total_row= df1.shape[0]
slicer = int(total_row*0.8)

In [ ]:
X=df1.drop(['id' ,'total_points' ],axis ='columns') 


In [ ]:
y_train = df1.loc[0:slicer, {'total_points'}]

y_test = df1.loc[slicer:, {'total_points'}]


In [ ]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
x_train = np.concatenate((np.ones((x_train.shape[0],1)), x_train), axis=1)
x_test = np.concatenate((np.ones((x_test.shape[0],1)), x_test), axis=1)



In [ ]:
x_test.shape

In [ ]:
x_train.shape

In [ ]:
theta = np.ones((x_train.shape[1],1))

In [ ]:
x_train.shape

In [ ]:
alpha = 0.001
itrs = 500

In [ ]:
def costfunction(X , Y , theta):
    X.shape[0]
    temp = np.square(np.matmul(X,theta)) - Y
    cost = (1/(2*X.shape[0])*np.sum(temp))
    return cost 

In [ ]:
plot_j_theta_train =[]
plot_j_theta_test =[]
itr = []

In [ ]:
m = x_train.shape[0]

In [ ]:
for i in range(itrs):
    gradient =np.matmul(np.transpose(x_train),np.matmul(x_train,theta)-y_train)
    theta = theta - (alpha/m) *gradient  
    J_theta_train = costfunction(x_train ,y_train , theta)
    J_theta_test = costfunction(x_test,y_test,theta)
    print(i , "train_cost_fxn" , J_theta_train, "test_cost_fxn" , J_theta_test)
    
    
    plot_j_theta_train.append(J_theta_train)
    plot_j_theta_test.append(J_theta_test)
    itr.append(i)
    
    
   

In [ ]:
plt.plot( itr, plot_j_theta_train)
plt.xlabel("iteration")
plt.ylabel("costfxn")


In [ ]:
x_test=X.loc[slicer:]
y_test = df.loc[slicer:, {'total_points'}]

In [ ]:
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
x_test = np.concatenate((np.ones((x_test.shape[0],1)) , x_test) , axis = 1 )

In [ ]:
prediction = np.matmul(x_test , theta)

In [ ]:
prediction

In [ ]:
prediction = (prediction*std)+ mean
y_test = (y_test*std)+mean

In [ ]:
prediction

In [ ]:
predicted_value = pd.DataFrame(prediction,columns = ['prediction'])

In [ ]:
predicted_value =predicted_value.to_numpy()

In [ ]:
error = ((predicted_value - y_test)/predicted_value)*100

In [ ]:
error

predicted_value

In [ ]:
y_test